# Zadanie 1

In [8]:
def pl_true_symbols(s, m):
    if isinstance(s, str):  
        if s in m:
            return m[s]
        else:
            raise ValueError(f"Symbol '{s}' nie znaleziony w przypisaniu m.")

    elif isinstance(s, tuple):  
        operator = s[0] 

        if operator == '~': # Negacja
            if len(s) != 2:
                raise ValueError(f"Operator ~ oczekuje jednego argumentu, otrzymano {len(s)-1} dla {s}")
            val = pl_true_symbols(s[1], m)
            return 1 - val

        elif operator == '&': # Koniunkcja (AND)
            if len(s) != 3:
                raise ValueError(f"Operator & oczekuje dwóch argumentów, otrzymano {len(s)-1} dla {s}")
            val1 = pl_true_symbols(s[1], m)
            val2 = pl_true_symbols(s[2], m)
            return 1 if val1 == 1 and val2 == 1 else 0

        elif operator == '|': # Alternatywa (OR)
            if len(s) != 3:
                raise ValueError(f"Operator | oczekuje dwóch argumentów, otrzymano {len(s)-1} dla {s}")
            val1 = pl_true_symbols(s[1], m)
            val2 = pl_true_symbols(s[2], m)
            return 1 if val1 == 1 or val2 == 1 else 0

        elif operator == '->': # Implikacja
            if len(s) != 3:
                raise ValueError(f"Operator -> oczekuje dwóch argumentów, otrzymano {len(s)-1} dla {s}")
            val1 = pl_true_symbols(s[1], m) 
            val2 = pl_true_symbols(s[2], m)
            
            # A -> B jest równoważne ~A | B
            return 1 if (1 - val1) == 1 or val2 == 1 else 0


        else:
            raise ValueError(f"Nieznany operator: {operator} w zdaniu {s}")
    else:
        raise TypeError(f"Nieprawidłowy format zdania S: {s}. Oczekiwano stringa lub krotki.")



### Przypadek 0: 0 spójników

In [9]:
m0 = {'p': 1}
s0 = 'p'
print(f"Zdanie: {s0}, Przypisanie: {m0}, Wynik: {pl_true_symbols(s0, m0)}")

Zdanie: p, Przypisanie: {'p': 1}, Wynik: 1


### Przypadek 1: 1 spójnik

In [10]:
m1a = {'p': 0}
s1a = ('~', 'p')
print(f"Zdanie: {s1a}, Przypisanie: {m1a}, Wynik: {pl_true_symbols(s1a, m1a)}")

Zdanie: ('~', 'p'), Przypisanie: {'p': 0}, Wynik: 1


In [11]:
m1b = {'p': 1, 'q': 1}
s1b = ('&', 'p', 'q')
print(f"Zdanie: {s1b}, Przypisanie: {m1b}, Wynik: {pl_true_symbols(s1b, m1b)}")

Zdanie: ('&', 'p', 'q'), Przypisanie: {'p': 1, 'q': 1}, Wynik: 1


### Przypadek 2: 2 spójniki

In [12]:
m2 = {'p': 0, 'q': 1, 'r': 0} 
s2 = ('->', ('|', 'p', ('~', 'q')), 'r')
print(f"Zdanie: {s2}, Przypisanie: {m2}, Wynik: {pl_true_symbols(s2, m2)}")

Zdanie: ('->', ('|', 'p', ('~', 'q')), 'r'), Przypisanie: {'p': 0, 'q': 1, 'r': 0}, Wynik: 1


In [13]:
m2_v2 = {'p': 1, 'q': 0, 'r': 0} 
s2_v2 = ('->', ('&', 'p', ('~', 'q')), 'r')
print(f"Zdanie: {s2_v2}, Przypisanie: {m2_v2}, Wynik: {pl_true_symbols(s2_v2, m2_v2)}")

Zdanie: ('->', ('&', 'p', ('~', 'q')), 'r'), Przypisanie: {'p': 1, 'q': 0, 'r': 0}, Wynik: 0


# Zadanie 2

In [11]:
# Funkcja wykonująca rezolucję dwóch klauzul
def resolve(c1, c2):
    resolvents = set()  # Zbiór na nowe klauzule 

    # Przechodzimy przez każdy literał w pierwszej klauzuli
    for lit in c1:
        # Tworzymy przeciwny literał 
        neg = '-' + lit if not lit.startswith('-') else lit[1:]

        # Jeśli przeciwny literał znajduje się w drugiej klauzuli, możemy zastosować rezolucję
        if neg in c2:
            # Tworzymy nową klauzulę bez tych dwóch przeciwnych liter
            new_clause = (c1 - {lit}) | (c2 - {neg})

            # Sprawdzamy, czy nowa klauzula nie jest tautologią (czy nie zawiera P12 i -P12 naraz)
            if not any(l in new_clause and ('-' + l) in new_clause for l in new_clause):
                resolvents.add(frozenset(new_clause))  # Dodajemy nową klauzulę jako frozenset

    return resolvents  # Zwracamy wszystkie nowe klauzule wygenerowane z tej pary

# Funkcja główna realizująca algorytm rezolucji
def pl_resolution(clauses, alpha):
    # Negujemy α (np. jeśli chcemy dowieść -P12, to dodajemy P12 do KB)
    neg_alpha = alpha[1:] if alpha.startswith('-') else '-' + alpha

    # Tworzymy kopię zbioru klauzul i dodajemy ¬α
    clauses = set(clauses)
    clauses.add(frozenset([neg_alpha]))

    print(f"Sprawdzanie: KB ⊨ {alpha}? (czy można to logicznie udowodnić?)\n")

    while True:
        new = set()  # Zbiór nowych klauzul w tej iteracji

        clause_list = list(clauses)  # Konwertujemy zbiór na listę, aby iterować po parach

        # Dla każdej pary klauzul generujemy rezolwenty
        for i in range(len(clause_list)):
            for j in range(i + 1, len(clause_list)):
                resolvents = resolve(clause_list[i], clause_list[j])

                # Jeśli wśród rezolwentów jest pusty zbiór → sprzeczność → udowodniono
                if frozenset() in resolvents:
                    print("  ❗ Wykryto sprzeczność (pusta klauzula).")
                    print(f"✅ KB ⊨ {alpha} — WNIOSEK UDOWODNIONY.")
                    return True

                # Dodajemy wszystkie nowe klauzule
                new |= resolvents

        # Jeśli nie ma nowych klauzul — nic więcej nie da się wywnioskować
        if new <= clauses:
            print("Brak nowych wniosków. Sprzeczność nie została znaleziona.")
            print(f"KB ⊭ {alpha} — WNIOSEK NIEUDOWODNIONY.")
            return False

        # Aktualizujemy bazę wiedzy o nowe klauzule
        clauses |= new

# ----------------------------
# Przykład bazy wiedzy (KB):
# ----------------------------
kb = [
    frozenset(['-B11', 'P12', 'P21']),  # Jeśli nie ma smoka w B11, to albo w P12, albo w P21
    frozenset(['-P12', 'B11']),         # Jeśli nie ma smoka w P12, to w B11 jest
    frozenset(['-P21', 'B11']),         # Jeśli nie ma smoka w P21, to w B11 jest
    frozenset(['-B11']),                # Nie ma smoka w B11
]

alpha = '-P12'  
result = pl_resolution(kb, alpha)
print("Wniosek udowodniony!" if result else "Wniosek nieudowodniony.")

Sprawdzanie: KB ⊨ -P12? (czy można to logicznie udowodnić?)

  ❗ Wykryto sprzeczność (pusta klauzula).
✅ KB ⊨ -P12 — WNIOSEK UDOWODNIONY.
Wniosek udowodniony!


# Zadanie 9

Oznaczenia (predykaty i stałe):

  Loves(x,y): x kocha y

  Kills(x,y): x zabija y

  Animal(y): y jest zwierzęciem

  Tuna: kot Tuna (stała)

  Jack, Jola: osoby (stałe)

  ∀, ∃: kwantyfikatory ogólny i szczegółowy
    
Wiedza ogólna (KB):
1. Każdy, kto kocha wszystkie zwierzęta, jest przez kogoś kochany.
    ∀x[(∀y(Animal(y)→Loves(x,y)))→(∃zLoves(z,x))]
2. Nikt nie kocha tego, kto zabija zwierzę.
    ∀x∀y∃z[(Animal(z)∧Kills(y,z))→¬Loves(x,y)]
3. Jack kocha wszystkie zwierzęta.
    ∀y(Animal(y)→Loves(Jack,y))
4. Albo Jack, albo Jola zabili kota Tuna.
    Kills(Jack,Tuna)∨Kills(Jola,Tuna)
5. Tuna jest zwierzęciem.
    Animal(Tuna)



In [14]:
from sympy import symbols, Or, And, Not
from sympy.logic.boolalg import to_cnf
from sympy.logic.inference import satisfiable

# Predykaty jako funkcje
Loves = lambda x, y: symbols(f"Loves_{x}_{y}")
Kills = lambda x, y: symbols(f"Kills_{x}_{y}")
Animal = lambda x: symbols(f"Animal_{x}")

# Stałe (osoby i kot)
Jack = "Jack"
Jola = "Jola"
Tuna = "Tuna"
Someone = "Someone"

Skolem1 = "Skolem1"

# KB: baza wiedzy w postaci CNF, po ręcznej skolemizacji

kb = And(
    # (1) Każdy, kto kocha wszystkie zwierzęta, jest przez kogoś kochany
    # Wersja po skolemizacji dla Jacka i Tuna:
    Or(Not(Loves(Jack, Tuna)), Loves(Skolem1, Jack)),

    # (2) Nikt nie kocha tego, kto zabija zwierzę
    # Zapisane dla Jacka i Joli oraz Tuna
    Or(Not(Animal(Tuna)), Not(Kills(Jack, Tuna)), Not(Loves("Someone", Jack))),
    Or(Not(Animal(Tuna)), Not(Kills(Jola, Tuna)), Not(Loves("Someone", Jola))),

    # (3) Jack kocha wszystkie zwierzęta
    # Dla Tuny: jeśli jest zwierzęciem, to Jack ją kocha
    Or(Not(Animal(Tuna)), Loves(Jack, Tuna)),

    # (4) Jack lub Jola zabili kota Tuna
    Or(Kills(Jack, Tuna), Kills(Jola, Tuna)),

    # (5) Tuna jest zwierzęciem
    Animal(Tuna),
    
    # (6) Ktoś (Someone) kocha Jacka – dodane, żeby wymusić konsekwencję
    Loves(Someone, Jack)
)

# Negacja wniosku: zakładamy, że Jola NIE zabiła Tuny
negated_conclusion = Not(Kills(Jola, Tuna))

# Połącz KB z negacją wniosku
full_formula = And(kb, negated_conclusion)

# Przekształć do CNF
cnf = to_cnf(full_formula, simplify=True)

# Sprawdzenie, czy formuła jest spełnialna (czyli: czy KB ∧ ¬wniosek jest spójna)
result = satisfiable(cnf)

# Interpretacja wyniku
if not result:
    print("Wniosek: Z KB logicznie wynika, że Jola zabiła kota Tuna.")
else:
    print("Wniosek: Nie można jednoznacznie stwierdzić, że Jola zabiła kota Tuna.")
    print("Spełniające przypisanie (przykład):", result)


Wniosek: Z KB logicznie wynika, że Jola zabiła kota Tuna.


Zdanie:

    „Jack kocha wszystkie zwierzęta” i „Nikt nie kocha tych, co zabijają zwierzęta”

nie zakazuje Jackowi zabijać zwierząt, tylko implikuje, że nikt go wtedy nie będzie kochał. Ale to nie daje sprzeczności, jeśli nikt go i tak nie kocha (np. Loves(Someone, Jack) = False).

W tym celu dodałem kaluzulę nr 6 - wymusza ona, że ktoś kocha Jacka